In [4]:
import sys
import random
sys.path.append("home/kelab/m6AAIpy2")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [6]:
paths = ('home/kelab/iM6A/mouseRAC10000_c{}.h5'.format(x) for x in range(1, 6))

In [7]:
models = [load_model(y, custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for y in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
models

### Read data

In [5]:
import numpy as np
import pandas as pd

In [2]:
Fasta = pd.read_csv("mm10_Fasta_Non_Donor_Acceptor_Replace.csv")

In [3]:
Fasta

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,LastIntronLength,SecondToLastLength,Pre,PreSequence,SecondToLastExonSequence,LastIntronSequence,LastExonSequence,Truncation,Seq,Replace
0,Pdcd11,chr19,+,47090765,47131143,47092798,47130810,36,"47090765,47092781,47093750,47095207,47096854,4...","47090867,47092900,47093882,47095375,47097016,4...",...,525,166,39182,AGAACGGAAGTAGCCCTGGTCTTACCTACGCTCGTGGACGAGTCGC...,ATGTGGATGTGATCGTCAAGTTTGCCCAGCTGGAGTTCCAGCTCGG...,GTGAGGGCGACAGCAGGGCTCATTCCTGTTGGAGTGTGGGCTGGGA...,GGACATCTTTGAGCGGGTCATTCACCTGAGCCTGGCGCCTAAGAGG...,GTGAGGGCGACAGCAGGGCTCATTCCTGTTGGAGTGTGGGCTGGGA...,ATGTGGATGTGATCGTCAAGTTTGCCCAGCTGGAGTTCCAGCTCGG...,CCTTAAACTTTTAATTCTACTTGCACCGATTAGCTAGTTAGAGCAG...
1,Trim80,chr11,+,115440544,115448270,115440553,115448220,8,"115440544,115440706,115441080,115445015,115445...","115440606,115440750,115441745,115445111,115445...",...,843,63,6213,AGAGAGACCATGAAACAGCGGCCACAGAGAAAGGAGAGAAGACGGC...,GAATCAGGATGAGCTCCTATGAGGTGCTGCCCGGAACTGTGGAGAG...,GTGAGTTCTCAAACCAGAGGCCAACCCTGGGGGAGGAGAGGAGGGA...,ATTACTGCAACCTGAACTTTGACCCCTGCACGGCCAGCGAAGAGCT...,GTGAGTTCTCAAACCAGAGGCCAACCCTGGGGGAGGAGAGGAGGGA...,GAATCAGGATGAGCTCCTATGAGGTGCTGCCCGGAACTGTGGAGAG...,CGCCTCTGGGAATTTGACAATGATGTTGTCCAGCTTGTTTACATTG...
2,Strbp,chr2,-,37569867,37647285,37584029,37647118,17,"37569867,37586509,37590656,37596703,37599120,3...","37584106,37586554,37590780,37596844,37599255,3...",...,2403,45,60731,GTGTATTCCAGAGAACAATTTTGAGATATCTTTGCTTATATTTGAC...,GTTACGGAACCCCATATGGTTACAGCACAGCTGCCCCTGCATATG,GTATGTACACTTCCTTACTGATTCCTCCTCTTAGCAAATTTGTCAC...,GTTTGCCCAAGAGAATGGTTCTGTTACCCGTTATGAAATTCCCAAC...,GTATGTACACTTCCTTACTGATTCCTCCTCTTAGCAAATTTGTCAC...,GTTACGGAACCCCATATGGTTACAGCACAGCTGCCCCTGCATATGC...,CTCGTGTCCACAGGCGCCCAGCATCCACACCCAGAGGCAAAGAAAA...
3,Kl,chr5,+,150952606,150993817,150952716,150991854,5,"150952606,150980609,150982502,150988392,150991...","150953541,150981120,150982771,150989494,150993...",...,2022,1102,35786,GATAATCATTGCTCGTGGGGCGGCGGGAGCGGGGGTGGGCACCGCG...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,CCTACGTGTTGGACGACATCAACCTTTGTGGCTACTTTGCGTATTC...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...,GATGCAAGTCGCTGTTTGATTGGCTTTTAGAAAATGCCCAGAACAG...
4,Arhgap21,chr2,-,20847918,20967721,20848711,20967410,25,"20847918,20853037,20854934,20855353,20855580,2...","20850385,20853069,20855040,20855395,20855706,2...",...,2652,32,114652,GGAAGGCTCTCCGAAAGCTGATTTTTCTCTGGCCTCCGAGTAACAC...,ATTCAGCTACTAGTGACTCCGCAAAATCTAAG,GTAAGTTGTTCTATACAGCTGGAAAACACTTCGTGGTAGCTTTGGG...,GGCTCCTGGGGATCCGGGAAGGATCAGTATAGCAGGGAACTGCTTG...,GTAAGTTGTTCTATACAGCTGGAAAACACTTCGTGGTAGCTTTGGG...,ATTCAGCTACTAGTGACTCCGCAAAATCTAAGCAAAGTTGTTCTAT...,CTTTAATAGTACACCATCAACAGGCATGTATCATTTTGTCATCAGT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179,Wnt4,chr4,+,137277488,137299726,137277679,137296779,5,"137277488,137289113,137295440,137295659,137296...","137277756,137289349,137295572,137295802,137299...",...,509,143,18171,GGCTCTGGGGCGGCGCTGACAGTCGGGTCCGCCGGGCAGCGGGCGC...,GCTTCCAGTGGTCAGGATGCTCGGACAACATCGCCTATGGCGTAGC...,GTAGAGTGGTACCAACCCCATCGACACTGGGGCAGGGCTGCACGGG...,GCCATCTTGACACACATGCGGGTGGAGTGCAAGTGTCACGGGGTGT...,GTAGAGTGGTACCAACCCCATCGACACTGGGGCAGGGCTGCACGGG...,GCTTCCAGTGGTCAGGATGCTCGGACAACATCGCCTATGGCGTAGC...,AAATGTTTGTACATTATTAATATACATTTACAATATTTATGCCTTG...
2180,Inpp4a,chr1,+,37299890,37410736,37357918,37408259,26,"37299890,37357342,37357814,37358507,37364109,3...","37300118,37357404,37358024,37358552,37364228,3...",...,9206,155,98875,GCGGGGTTGTGCGCTGTGTTCTGAGAGAAGGCGGTGGCAGGATTCG...,GTCTGTCGCCGCCTTAACGGGGTCCGATTCACCAGCTGCAAGAGTG...,GTGAGTGAGGCTGCTGCCAGGCTGCATCTGCCCAGCCACTTGCTCT...,TGAGGGTTGTCGGCGAGAAAACACAATGAAGAATGTTGGAAGTCGC...,GTGAGTGAGGCTGCTGCCAGGCTGCATCTGCCCAGCCACTTGCTCT...,GTCTGTCGCCGCCTTAACGGGGTCCGATTCACCAGCTGCAAGAGTG...,GAGGACAAGAAGTAGCGGTCCTTTATTTATAAGTGCTTATCCAGTA...
2181,Trps1,

### Select positive strand

In [4]:
Fasta_Pos = Fasta[Fasta["strand"]=="+"]

In [5]:
Fasta_Pos = Fasta_Pos.reset_index(drop = True)

In [6]:
Fasta_Pos

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,LastIntronLength,SecondToLastLength,Pre,PreSequence,SecondToLastExonSequence,LastIntronSequence,LastExonSequence,Truncation,Seq,Replace
0,Pdcd11,chr19,+,47090765,47131143,47092798,47130810,36,"47090765,47092781,47093750,47095207,47096854,4...","47090867,47092900,47093882,47095375,47097016,4...",...,525,166,39182,AGAACGGAAGTAGCCCTGGTCTTACCTACGCTCGTGGACGAGTCGC...,ATGTGGATGTGATCGTCAAGTTTGCCCAGCTGGAGTTCCAGCTCGG...,GTGAGGGCGACAGCAGGGCTCATTCCTGTTGGAGTGTGGGCTGGGA...,GGACATCTTTGAGCGGGTCATTCACCTGAGCCTGGCGCCTAAGAGG...,GTGAGGGCGACAGCAGGGCTCATTCCTGTTGGAGTGTGGGCTGGGA...,ATGTGGATGTGATCGTCAAGTTTGCCCAGCTGGAGTTCCAGCTCGG...,CCTTAAACTTTTAATTCTACTTGCACCGATTAGCTAGTTAGAGCAG...
1,Trim80,chr11,+,115440544,115448270,115440553,115448220,8,"115440544,115440706,115441080,115445015,115445...","115440606,115440750,115441745,115445111,115445...",...,843,63,6213,AGAGAGACCATGAAACAGCGGCCACAGAGAAAGGAGAGAAGACGGC...,GAATCAGGATGAGCTCCTATGAGGTGCTGCCCGGAACTGTGGAGAG...,GTGAGTTCTCAAACCAGAGGCCAACCCTGGGGGAGGAGAGGAGGGA...,ATTACTGCAACCTGAACTTTGACCCCTGCACGGCCAGCGAAGAGCT...,GTGAGTTCTCAAACCAGAGGCCAACCCTGGGGGAGGAGAGGAGGGA...,GAATCAGGATGAGCTCCTATGAGGTGCTGCCCGGAACTGTGGAGAG...,CGCCTCTGGGAATTTGACAATGATGTTGTCCAGCTTGTTTACATTG...
2,Kl,chr5,+,150952606,150993817,150952716,150991854,5,"150952606,150980609,150982502,150988392,150991...","150953541,150981120,150982771,150989494,150993...",...,2022,1102,35786,GATAATCATTGCTCGTGGGGCGGCGGGAGCGGGGGTGGGCACCGCG...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,CCTACGTGTTGGACGACATCAACCTTTGTGGCTACTTTGCGTATTC...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...,GATGCAAGTCGCTGTTTGATTGGCTTTTAGAAAATGCCCAGAACAG...
3,Ky,chr9,+,102505749,102546239,102506142,102542781,11,"102505749,102509110,102512972,102521288,102523...","102506278,102509173,102513035,102521363,102523...",...,2697,191,33248,TCTTAGGGGATAGATGGTGGCTATGAGGCAGCACTCAGCCCAGCCA...,CTACAATGAATTCTACTTCCTCACCCACCCCGCCCTGTTCATCGAG...,GTAAGGAGAAGCACAAGGCGGAGCCCCAGTGACGGCTGAAGCTCCC...,TCAATGGGAAAGCCACAATCACCATCGAGAGCCGGGCCCCAACACT...,GTAAGGAGAAGCACAAGGCGGAGCCCCAGTGACGGCTGAAGCTCCC...,CTACAATGAATTCTACTTCCTCACCCACCCCGCCCTGTTCATCGAG...,ACATCATACCTTTTCTTAAGTGAAGCCCTGGCACTTTATAGCAGAG...
4,Zdbf2,chr1,+,63273264,63314576,63290737,63309945,7,"63273264,63275095,63277930,63280766,63290676,6...","63273472,63275178,63277983,63280877,63290782,6...",...,7593,131,21651,GCCCAGTCGCGACTCGGAGCCCGCGTCGAGCGACACGcgctccgct...,CACTTGTTTAGTTCTCAGCACAGAAGCCTGACCAGACAGAGCAGAC...,GTAAGATAGTTCAGTGGAATGTTAATATCTCTGTCAGTGGTATATT...,ATCTGCACCAAATGAGCCAGAGgcggcggcagcagcggcagcagca...,GTAAGATAGTTCAGTGGAATGTTAATATCTCTGTCAGTGGTATATT...,CACTTGTTTAGTTCTCAGCACAGAAGCCTGACCAGACAGAGCAGAC...,AGGAGAACAATCTGCTGAAGACGCTGGCCAGTCCGGAGCAGCTCGC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082,Dlg3,chrX,+,100767722,100818408,100768071,100816325,20,"100767722,100771306,100771614,100772056,100772...","100768428,100771360,100771665,100772181,100772...",...,919,92,47485,TTGACTCCACTCAGATCCATCCTGGGGACCTGAGCAGAAGAGGTTC...,GGAAATGAACCGACGGCAGACATACGAACAAGCAAATAAGATCTTT...,GTAAGGATCCGGAGTCCTCTGGGCAGGCCAGATGCGAGCTCAGGGA...,CCATTGTACAGGGTGACTCACTGGAAGAGATTTATAACAAAATCAA...,GTAAGGATCCGGAGTCCTCTGGGCAGGCCAGATGCGAGCTCAGGGA...,GGAAATGAACCGACGGCAGACATACGAACAAGCAAATAAGATCTTT...,CTCTACAACCAGAACCTCTCCAACAGGCCTTCTTTCAATCGAACCC...
1083,Fam117b,chr1,+,59913005,59985346,59913163,59981727,8,"59913005,59949052,59952934,59958321,59968953,5...","59913752,59949204,59953024,59958435,59969097,5...",...,6405,121,61877,GCGGCTGCATCACCTCGTCGGTGCCCGTCGCTGCCCGCGCGCCCCC...,AGAATGGAAACAATTCTCCTTTGCCAAAATATGCAACCTCACCAAA...,GTAAGTACCCCACATAAACACACTTTTTACATTAGAATTTTCAAAT...,GCCAAAACAACTTCATGAGATTCCAGCCTTTTACTGTCCTGACAAA...,GTAAGTACCCCACATAAACACACTTTTTACATTAGAATTTTCAAAT...,AGAATGGAAACAATTCTCCTTTGCCAAAATATGCAACCTCACCAAA...,GACACCTTCCTCATGGAGATGCTGGAGGCACCACATCAAGCC

In [16]:
PreSequence = Fasta_Pos["PreSequence"].tolist()
Sequence = Fasta_Pos["Seq"].tolist()
ReplaceSequence = Fasta_Pos["Replace"].tolist()

In [17]:
df = pd.DataFrame(range(-100,1001),columns=["index"])
for i in range(len(Fasta_Pos)):
    Pre = PreSequence[i]
    Seq = Sequence[i]
    Replace = ReplaceSequence[i]
    
    # Define seq
    SecondToLastExon = Seq[0:(Fasta_Pos.loc[i,"SecondToLastLength"])]
    Intron = Seq[(Fasta_Pos.loc[i,"SecondToLastLength"]):(Fasta_Pos.loc[i,"SecondToLastLength"]+100)]
    LastExon = Seq[-(Fasta_Pos.loc[i,"LastExonLength"]):]   

    # sequence scramble    
    Replace = list(Replace)
    random.shuffle(Replace)
    Replace = "".join(Replace)       
    
    input_sequence = Pre + SecondToLastExon + Replace + LastExon    

    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    m6AAI_prob = m6AAI_prob.tolist()
    m6AAI_prob = m6AAI_prob[-(100 + len(LastExon)):]


    iM6A_prob = m6AAI_prob
    
    Start = 100
    Pre = iM6A_prob[0:Start]
    Last = iM6A_prob[Start:]
    
    if len(Last) < 1000:
        Last = Last + [0]*(1000-len(Last))
    if len(Last) >= 1000:
        Last = Last[0:1000]
    
    New = np.array(Pre + Last)
    New = pd.DataFrame({Fasta_Pos.loc[i,"name"]:New})
    df = pd.merge(df, New, left_index=True, right_index=True)        

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


### Select negative strand

In [18]:
Fasta_Neg = Fasta[Fasta["strand"]=="-"]

In [19]:
Fasta_Neg = Fasta_Neg.reset_index(drop = True)

In [20]:
Fasta_Neg

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,Group,Sequence,GeneLength,LastExonLength,LastIntronLength,SecondToLastLength,Pre,PreSequence,LastIntronSequence,LastExonSequence
0,Adora1,chr1,-,134199224,134235431,134202950,134234355,3,"134199224,134234014,134235227,","134203590,134234412,134235431,",...,2,ACTCAGCTCCCGCGCCGGGGGCGACGAAGGCGCCGGCGCCAGAGTG...,36207,4366,30424,398,1417,ACTCAGCTCCCGCGCCGGGGGCGACGAAGGCGCCGGCGCCAGAGTG...,GTGAGTTCATAGCAGCCCAAGGTACTCTGTGCAACCCTGTGTGGGG...,GTACAAGACAGTGGTGACTCAGCGGCGGGCAGCAGTGGCCATAGCC...
1,Sntg1,chr1,-,8361474,9299238,8363474,8803943,21,"8361474,8414202,8445026,8447979,8463393,846543...","8363633,8414313,8445119,8448132,8463465,846555...",...,2,CTTCGCGGCGATACAGCCTCTCGGAAACAAAGGTATCCCTAGCCTT...,937764,2159,50569,111,885036,CTTCGCGGCGATACAGCCTCTCGGAAACAAAGGTATCCCTAGCCTT...,GTAAACACAAGAAACCTCCATATAATACATCTCCAATTATGTCAGT...,GAGTTGGAATTTTCAAATTTATTTGCTGTTCTTCACTGTATTCATT...
2,Adgrb3,chr1,-,25067549,25829707,25068167,25826760,31,"25067549,25074684,25084243,25093764,25098875,2...","25068356,25074789,25084279,25094408,25098915,2...",...,2,GGCTCTGGCCGCTGGATACCACCGCCCCCCCCCCTATTATTTTTTT...,762158,807,6328,105,755023,GGCTCTGGCCGCTGGATACCACCGCCCCCCCCCCTATTATTTTTTT...,GTGAGTATATTCTGCATCCACAATGCCCCAAGCCAAGCTGGGCCAC...,GAAAACCCAGCACCAAACAAGAATCCATGGGACACTTTCAAACCCC...
3,Prim2,chr1,-,33453809,33669795,33454085,33669011,14,"33453809,33464052,33480457,33484681,33512015,3...","33454304,33464121,33480540,33484808,33512201,3...",...,2,CTTCCCGTTCCAAATGAACTCTCCCGCCACTGGGAGAGAGCGGCTG...,215986,495,9748,69,205743,CTTCCCGTTCCAAATGAACTCTCCCGCCACTGGGAGAGAGCGGCTG...,GTAAGTAAATTGTTTAACTTTATAGATTGGTTGTATGTGGCTTGTT...,GTGGACGATTGTGGCTTTTCTTTGAATCATCCAAATCAGTTCTTTT...
4,Pld5,chr1,-,175962305,176275298,175963827,176274874,10,"175962305,175970477,175975458,175984446,175993...","175964084,175970592,175975627,175984583,175994...",...,2,AGTTGGTGCGGAGCAGCCTCGAGCGGAGCTCCCGCTCGGGACCGCA...,312993,1779,6393,115,304821,AGTTGGTGCGGAGCAGCCTCGAGCGGAGCTCCCGCTCGGGACCGCA...,GTAGGTGTGGGGGTTGAGAAGCCCCTCGGGGGTCACAGTAGGGGGA...,GAAATTTTGACTGGGTAGGGAATGACTTTACCCAGAACGCTGGCAC...
5,Syt14,chr1,-,192891232,193035775,192897306,193035635,8,"192891232,192901751,192930477,192933113,192980...","192897620,192901941,192930927,192933385,192980...",...,2,GCCCCGCCCCAGCTCCTGGAGGCCCAGCTGCCTGGCCCCGGGGCGT...,144543,6388,4131,190,134024,GCCCCGCCCCAGCTCCTGGAGGCCCAGCTGCCTGGCCCCGGGGCGT...,GTGAGTGAAAAAGAGTTATTTTTATCTTAATGTTTTCGGCACTTAG...,ATACATATGTTAAGTTAACACTGCTGAATTCCATGGGCCAAGAGAT...
6,Cpa6,chr1,-,10324726,10719945,10325123,10719727,11,"10324726,10329472,10337266,10408268,10409256,1...","10325311,10329557,10337469,10408359,10409367,1...",...,2,GCTTCCAGGCTGGGCTGGGCTGCATTTCCTCACAAGACCCAGTCTT...,395219,585,4161,85,390473,GCTTCCAGGCTGGGCTGGGCTGCATTTCCTCACAAGACCCAGTCTT...,GTAAGTTTCCTGTGCACGTCATATTAGAATTGTGCCATGACTTCCC...,ATGTAAGCTCTGGTAACTCAATGGATTGGGCTTATAAAAATGGAAT...
7,Lonrf2,chr1,-,38794508,38821215,38798028,38821215,11,"38794508,38800650,38804238,38807040,38807835,3...","38798223,38800800,38804401,38807199,38807957,3...",...,2,ATGAAGAACTATGATCAAGCCCTGCAGGATGCTGAAGCAGTTTGTC...,26707,3715,2427,150,20565,ATGAAGAACTATGATCAAGCCCTGCAGGATGCTGAAGCAGTTTGTC...,GTAGGCTTTCCTTCCTTCTTTCCTGGTGCAGCTCCTCCCCTCAGTC...,AGCAATTCTAGTGGCCCTGCCTGGTCCTGGTGGATTCTGGCCGTGT...
8,Dnah7a,chr1,-,53397005,53706761,53397106,53706692,66,"53397005,53405675,53411505,53413731,53419544,5...","53397313,53405779,53411683,53413884,53419775,5...",...,2,CTCCTCTCCAGGTCCTGAGTTCCGCAGCAGCGATCCCGGCCGGCGT...,309756,308,8362,104,301086,CTCCTCTCCAGGTCCTGAGTTCCGCAGCAGCGATCCCGGCCGGCGT...,GTTAGTATTTACTAAGGGTTACATGCACGTGTGTGCGGATATGGAT...,ATGTGGCTAAAGCCATGCAAAAAGTCAGACATACCCAGACGACCAA...
9,Pgap1,chr1,-,54472993,54557682,54480705,54557580,27,"54472993,54481851,54486469,54490220,54492028,5...","54480844,54481956,54486657,54490271,54492164,5...",...,2,AGGCGCGCGCTGCTCAGCTGGGCACGGCTTGGCACCGGAGTCTTTC...,84689,7851,1007,105,75831,AGGCGCGCGCTGCTCAGCTGGGCACGGCTTGGCACCGGAGTCTTTC...,GTACGACAG

In [21]:
PreSequence = Fasta_Neg["PreSequence"].tolist()
Sequence = Fasta_Neg["Seq"].tolist()
ReplaceSequence = Fasta_Neg["Replace"].tolist()

In [22]:
for i in range(len(Fasta_Neg)):
    Pre = PreSequence[i]
    Seq = Sequence[i]
    Replace = ReplaceSequence[i]
    
    # Define seq
    SecondToLastExon = Seq[0:(Fasta_Neg.loc[i,"SecondToLastLength"])]
    Intron = Seq[(Fasta_Neg.loc[i,"SecondToLastLength"]):(Fasta_Neg.loc[i,"SecondToLastLength"]+100)]
    LastExon = Seq[-(Fasta_Neg.loc[i,"LastExonLength"]):] 
    
    # sequence scramble
    Replace = list(Replace)
    random.shuffle(Replace)
    Replace = "".join(Replace)        

    input_sequence = Pre + SecondToLastExon + Replace + LastExon    

    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    m6AAI_prob = m6AAI_prob.tolist()
    m6AAI_prob = m6AAI_prob[-(100 + len(LastExon)):]

    iM6A_prob = m6AAI_prob
    
    Start = 100
    Pre = iM6A_prob[0:Start]
    Last = iM6A_prob[Start:]
    
    if len(Last) < 1000:
        Last = Last + [0]*(1000-len(Last))
    if len(Last) >= 1000:
        Last = Last[0:1000]
    
    New = np.array(Pre + Last)
    New = pd.DataFrame({Fasta_Neg.loc[i,"name"]:New})
    df = pd.merge(df, New, left_index=True, right_index=True)        

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


In [23]:
df

,index,Gpr39,Lmx1a,A830018L16Rik,Efhc1,Khdrbs2,Gpr45,Dnah7b,Plcl1,Bmpr2,...,Calhm1,Calhm3,Obfc1,Xpnpep1,Gpam,Dclre1a,Vax1,Pdzd8,Rab11fip2,Sfxn4
0,-1000,3.404065e-07,9.960404e-10,4.522095e-05,9.874284e-09,1.628747e-08,0.000000e+00,1.243748e-11,1.650154e-10,1.871058e-08,...,0.0,0.0,3.586399e-10,8.881744e-11,1.051486e-09,2.838641e-07,0.0,6.651790e-09,3.535140e-11,2.615876e-12
1,-999,1.569549e-07,3.223006e-09,1.120975e-10,2.418615e-04,5.115084e-09,0.000000e+00,2.661461e-11,1.055594e-08,1.917069e-08,...,0.0,0.0,2.415429e-11,2.263511e-11,3.545020e-05,2.495632e-07,0.0,9.390646e-10,3.356898e-11,2.372894e-12
2,-998,1.477434e-06,1.463316e-08,8.310132e-11,2.065359e-08,6.443544e-11,0.000000e+00,2.903487e-11,1.031167e-08,1.469893e-03,...,0.0,0.0,8.063296e-11,4.901663e-10,9.865229e-10,5.832165e-09,0.0,3.882183e-09,3.725036e-11,5.243514e-12
3,-997,3.909261e-02,1.884429e-08,2.538651e-11,3.167930e-09,6.628977e-11,0.000000e+00,1.204637e-11,2.273915e-09,6.978952e-07,...,0.0,0.0,4.888764e-11,1.206728e-11,2.855645e-11,2.476456e-08,0.0,9.266238e-10,2.242917e-11,1.832595e-10
4,-996,1.726828e-07,1.437113e-09,3.093705e-12,3.532801e-09,6.096118e-10,0.000000e+00,2.609395e-11,1.022763e-09,9.548468e-08,...,0.0,0.0,1.728234e-11,6.390974e-10,1.668099e-09,2.115764e-08,0.0,2.090651e-09,8.354505e-11,1.386252e-11
5,-995,9.388958e-09,1.655192e-08,6.037919e-12,9.071729e-05,1.326088e-09,0.000000e+00,5.621475e-12,2.080592e-09,2.905442e-08,...,0.0,0.0,1.074417e-09,4.098923e-11,2.406074e-10,2.221961e-07,0.0,9.432033e-08,3.808462e-10,9.303265e-12
6,-994,1.171199e-07,2.912713e-09,3.691158e-11,7.446640e-10,1.142202e-10,0.000000e+00,5.843972e-12,3.051843e-08,1.034898e-06,...,0.0,0.0,2.066749e-10,9.513816e-12,2.076795e-10,1.551780e-07,0.0,4.199840e-09,1.408965e-10,1.351011e-10
7,-993,5.527808e-07,1.172541e-09,3.363361e-11,3.861669e-10,3.404300e-11,0.000000e+00,2.968146e-11,2.367728e-09,2.675891e-07,...,0.0,0.0,2.915557e-10,3.583345e-11,2.084177e-10,6.301884e-08,0.0,5.924103e-09,8.081099e-10,2.065893e-10
8,-992,1.174919e-08,4.891917e-08,3.965262e-11,2.268177e-09,6.836415e-11,0.000000e+00,9.018527e-13,1.058769e-09,6.224791e-08,...,0.0,0.0,1.520923e-10,7.838705e-11,7.793340e-11,3.670696e-06,0.0,1.782334e-03,1.824507e-09,1.158569e-05
9,-991,4.336358e-08,2.968143e-02,6.295857e-13,1.774522e-10,5.798946e-10,0.000000e+00,3.935670e-12,1.385849e-08,1.764878e-07,...,0.0,0.0,9.794786e-10,2.475490e-11,9.185561e-11,1.372521e-01,0.0,9.884745e-08,1.174014e-10,3.979430e-10


In [ ]:
df.to_csv("iM6A_Mouse_LastIntronEnd_TruncationWithSpliceMutation_IntronReplaceWithShuffleExon_2genes.csv", index=0)

In [24]:
df.drop(["index"], axis=1, inplace=True)
df = df.T

In [25]:
df.loc["Sum"] = df.sum()
df = df.T

In [26]:
Value = df[["Sum"]]

In [27]:
Value.head(5)

,Sum
0,4.281457
1,4.887684
2,6.880113
3,3.894987
4,4.113095


In [28]:
df.drop(["Sum"], axis=1, inplace=True)
df = df.T

In [29]:
df.loc["Number"] = (df > 0).sum()
df = df.T

In [30]:
Number = df[["Number"]]

In [31]:
Number.head(5)

,Number
0,6017.0
1,6026.0
2,6031.0
3,6036.0
4,6039.0


In [32]:
Data = pd.concat([Value, Number], axis=1)

In [33]:
Data["Mean"] = Data["Sum"]/Data["Number"]

In [34]:
Data

,Sum,Number,Mean
0,4.281457,6017.0,0.000712
1,4.887684,6026.0,0.000811
2,6.880113,6031.0,0.001141
3,3.894987,6036.0,0.000645
4,4.113095,6039.0,0.000681
5,5.037679,6043.0,0.000834
6,5.462272,6046.0,0.000903
7,5.140427,6049.0,0.000850
8,4.697464,6055.0,0.000776
9,7.534130,6059.0,0.001243


In [ ]:
Data.to_csv("iM6A_Mouse_LastIntronEnd_TruncationWithSpliceMutation_IntronReplaceWithShuffleExon.csv", index=0)